In [14]:
import pandas as pd
import re
# util functions
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)


# PRODUCTS database

In [28]:
import pandas as pd
import re
fields = ['EAN',
    #'ITEM_NAME',
    'ITEM_DESCRIPTION',
    #'CATEGORY',
    #'SUB_CATEGORY',
    #'SUB_SUB_CATEGORY',
    #'BRAND_NAME',
    #'SKU',
    #'BUY_UNIT',
    #'DESCRIPTION_STATUS',
    #'ORGANIC_ITEM',
    #'KIRLAND_ITEM',
    #'FINELINE_NUMBER',
    #'ITEM_IMG',
]
df = pd.read_csv('PRODUCTS_orig.csv', sep='|', index_col=False, usecols=fields)
# sort
df.sort_values("EAN", inplace=True)
# dropping ALL duplicte values 
df.drop_duplicates(subset="EAN", keep=False, inplace=True)
# clean html tags
df['ITEM_DESCRIPTION'] = df['ITEM_DESCRIPTION'].apply(lambda x: re.sub('<[^<]+?>', '', x))
# filter EAN 8 length
df.EAN = df.EAN.apply(str)
df = df[df['EAN'].apply(lambda x: len(x)>=8)]
df = df[df.ITEM_DESCRIPTION.notnull()]
# get size, weight, quantity
# (\d*\.?\d+)\s*(gr?|kgs?)
df.describe()

,EAN,ITEM_DESCRIPTION
count,250865,250865
unique,250865,206011
top,780681010090,QUESO 1 KG.
freq,1,599


In [2]:
# store CATEGORY,SUB_CATEGORY,SUB_SUB_CATEGORY and BRAND_NAME in CSVs and delete their columns
#CATEGORY = pd.Series(df.CATEGORY.unique()).sort_values()
#CATEGORY.to_csv("category.csv")
#SUB_CATEGORY = pd.Series(df.SUB_CATEGORY.unique()).sort_values()
#SUB_CATEGORY.to_csv("sub_category.csv")
#SUB_SUB_CATEGORY = pd.Series(df.SUB_SUB_CATEGORY.unique()).sort_values()
#SUB_SUB_CATEGORY.to_csv("sub_sub_category.csv")
#BRAND_NAME = pd.Series(df.BRAND_NAME.unique()).sort_values()
#BRAND_NAME.to_csv("brand.csv")
#len(BRAND_NAME)
# delete unecessary columns
#del df['CATEGORY']
#del df['SUB_CATEGORY']
#del df['SUB_SUB_CATEGORY']
#del df['BRAND_NAME']
#df_country = pd.read_csv('gs1_countries.csv', sep='|', index_col=False)
#df_country.country_code = df_country.country_code.apply(str)
# get counrty code
#df['country_code'] = df['EAN'].str[:3]
# get csv of countries


# PRODUCTOS compra mercado libre

In [29]:
import pandas as pd
import re
df1 = pd.read_csv('productos_ml1.csv', index_col=False)
# dropping ALL duplicte values 
df1.drop_duplicates(subset="EAN", keep=False, inplace=True)
# sort
#df1.sort_values("EAN", inplace=True)
# filter EAN 8 length
df1.EAN = df1.EAN.apply(str)
df1 = df1[df1['EAN'].apply(lambda x: len(x)>=8)]
df1 = df1[df1.producto.notnull()]
df1.describe()

,EAN,producto
count,20181,20181
unique,20181,19901
top,7501008761229,BOLSA REGALO
freq,1,25


# OPEN FOODS FACTS # https://world.openfoodfacts.org/data

In [43]:
import pandas as pd
import re
fields = ['code',
 'product_name',
 #'quantity',
 #'brands',
 #'categories_en',
 #'countries_en',
 'image_url',
]
#df2 = pd.read_csv('openfoodfacts.org.products_reduced.csv', sep='\t', index_col=False, usecols=fields)
df2 = pd.read_csv('openfoodfacts.org.products_orig.csv', sep='\t', index_col=False, usecols=fields)
df2.rename(columns={'code': 'EAN'}, inplace=True)
# dropping ALL duplicte values 
df2.EAN = df2.EAN.apply(str)
df2.drop_duplicates(subset="EAN", keep=False, inplace=True)
# filter EAN 8 length
df2 = df2[df2['EAN'].apply(lambda x: len(x)>=8)]
df2 = df2[df2.product_name.notnull()]
df2.describe()

/home/maugb/.virtualenvs/test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,EAN,product_name,image_url
count,1567067,1567067,1158730
unique,1567067,1039877,1158662
top,3564700282434,Aceite de oliva virgen extra,https://static.openfoodfacts.org/images/produc...
freq,1,1206,30


# PRODUCTOS compra mercado libre 2

In [31]:
import pandas as pd
import re
fields = ['Clave_Producto',
    'Descripcion',
    #'Descripcion2',
    #'Precio_Compra',
    #'Precio_Minimo_Venta',
    #'Precio_Venta',
    #'Precio_Mayoreo',
    #'Precio_Medio_Mayoreo',
    #'Impuesto IVA',
    #'IEPS',
    #'Clave_Alterna',
    #'Catalogo SAT'
]
df3 = pd.read_csv('productos_202103.csv', index_col=False, usecols=fields)
df3.rename(columns={'Clave_Producto': 'EAN'}, inplace=True)
# dropping ALL duplicte values 
df3.drop_duplicates(subset="EAN", keep=False, inplace=True)
# filter EAN 8 length
df3.EAN = df3.EAN.apply(str)
df3 = df3[df3['EAN'].apply(lambda x: len(x)>=8)]
df3.Descripcion = df3['Descripcion'].apply(lambda x: " ".join(x.split(".")[1:]))
df3.Descripcion = df3['Descripcion'].apply(lambda x: x.split("(")[0])
df3.Descripcion = df3['Descripcion'].apply(lambda x: x.replace("* ", ""))
df3 = df3[df3.Descripcion.notnull()]
df3 = trim_all_columns(df3)
df3.describe()

,EAN,Descripcion
count,20640,20640
unique,20640,20217
top,7506425600397,
freq,1,30


In [44]:
df_join1 = pd.merge(df, df2, on='EAN', how='outer')
df_join1 = pd.merge(df3, df_join1, on='EAN', how='outer')
df_join1 = pd.merge(df1, df_join1, on='EAN', how='outer')
df_join1.describe()

,EAN,producto,Descripcion,ITEM_DESCRIPTION,product_name,image_url
count,1824292,20181,20640,250865,1567067,1158730
unique,1824292,19901,20217,206011,1039877,1158662
top,250498800000,BOLSA REGALO,,QUESO 1 KG.,Aceite de oliva virgen extra,https://static.openfoodfacts.org/images/produc...
freq,1,25,30,599,1206,30


In [45]:
#df_join1[df_join1.producto.isnull() & df_join1.ITEM_NAME.isnull()] # ITEM_NAME y producto son null
#df_join1[df_join1.producto.notnull() & df_join1.ITEM_NAME.notnull()].describe()
#df_join1[df_join1.producto.notnull()].describe()
f1 = (df_join1.producto.notnull() & df_join1.Descripcion.notnull())
f2 = (df_join1.producto.notnull() & df_join1.ITEM_DESCRIPTION.notnull())
f3 = (df_join1.producto.notnull() & df_join1.product_name.notnull())
f4 = (df_join1.Descripcion.notnull() & df_join1.ITEM_DESCRIPTION.notnull())
f5 = (df_join1.Descripcion.notnull() & df_join1.product_name.notnull())
f6 = (df_join1.ITEM_DESCRIPTION.notnull() & df_join1.product_name.notnull())
df_join1[f1 | f2 | f3 | f4 | f5 | f6].to_csv("test.csv")
